# 빅데이터분석개론 기말고사 2번
### 컴퓨터공학과 201835641 김지훈

# 1. 커피빈 매장 정보 크롤링

## 1.1 필요한 라이브러리 import

In [57]:
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

from selenium import webdriver
import time

## 1.2 크롤러 함수 선언

In [123]:
def CoffeeBean_store(result):
    CoffeeBean_URL = "https://www.coffeebeankorea.com/store/store.asp"
    wd = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
             
    for i in range(1, 370):  #매장 수 만큼 반복
        wd.get(CoffeeBean_URL)
        time.sleep(1)  #웹페이지 연결할 동안 1초 대기
        try:
            wd.execute_script("storePop2(%d)" %i)
            time.sleep(1) #스크립트 실행 할 동안 1초 대기
            html = wd.page_source
            soupCB = BeautifulSoup(html, 'html.parser')
            store_name_h2 = soupCB.select("div.store_txt > h2")
            store_name = store_name_h2[0].string
            print(store_name)  #매장 이름 출력하기
            store_info = soupCB.select("div.store_txt > table.store_table > tbody > tr > td")
            store_address_list = list(store_info[2])
            store_address = store_address_list[0]
            store_phone = store_info[3].string
            result.append([store_name]+[store_address]+[store_phone])
        except:
            continue 
    return

In [124]:
def main():
    result = []
    print('CoffeeBean store crawling >>>>>>>>>>>>>>>>>>>>>>>>>>')
    CoffeeBean_store(result)  #[CODE 1]
    
    CB_tbl = pd.DataFrame(result, columns=('store', 'address','phone'))
    CB_tbl.to_csv('김지훈-201835641-커피빈.csv', encoding='cp949', mode='w', index=True)


In [125]:
if __name__ == '__main__':
     main()

CoffeeBean store crawling >>>>>>>>>>>>>>>>>>>>>>>>>>
차병원점
강남대로점
청담에스점
신사점
역삼점
양재스포타임점
청담성당점
영동점
도곡점
영동고앞점
압구정시티점
서초우성점
논현팍스타워점
삼성오크우드점
트레이드타워점
삼성봉은사거리점
잠실신천점
서초지파이브점
논현동수면센터점
삼성전자강남사옥점
삼성생명강남사옥점
방배카페골목점
역삼포스틸타워뒷점
테헤란로하이닉스뒷점
삼성봉은사로점
방이점
매봉역점
남부터미널역점
강남역랭기지타워점
도심공항타워점
청담카페점
광화문점
경희궁의아침점
홍대역점
순화점
종로관철동점
소공동삼화빌딩점
강변테크노마트점
외대앞점
고대안암역점
순화동라마다호텔점
신촌점
건대스타시티점
하나은행경복궁역점
서울스퀘어점
목동점
여의도34번지점
여의도교보증권점
여의도순복음교회옆점
부산광복동점
대전탄방동점
일산웨스턴돔1호점
분당서현점
분당플라자점
분당차병원점
분당수내점
동탄라마다호텔점
창원상남동점
청주지웰시티점
무교CBTL점
국기원사거리점
무교점
남대문점
대학로마로니에점
인사점
테헤란로비젼타워점
서울대입구역점
코엑스인터점
뉴압구정점
디큐브시티점
현대자동차여의도점
선릉로93길점
강남역먹자골목점
삼성로점
선릉로86길점
안국역점
뉴교보타워옆점
IFC MALL2호점
상암SBS미디어센터점
마리오아울렛점
현대자동차남양연구소점
서초뱅뱅사거리점
현대자동차성내점
수원인계점
일산원마운트점
남양연구소C1식당점
여의도교보증권카페점
압구정현대아파트점
역삼역4번출구점
부평문화의거리점
강남대로센터점
구로G-ValleyMall
내방역7번출구유중아트점
도심공항점
타임스퀘어점
차병원사거리점
파르나스몰점
광화문역1번출구점
남양연구소PT동점
현대프리미엄아울렛김포점
여의도신영증권별관점
이태원제일기획옆점
강남역세계빌딩점
광교아브뉴프랑점
광장동대한제지점
신천역점
인천공항탑승동130gate앞점
석촌호수점
오목교역사거리점
이천하이닉스점
송도홈플러스점
올림픽공원북2문앞점
삼성역글라스타워뒷점
범계역로데오점
포스코옆삼성로86길점
상암한국지역정보개발원점


ProtocolError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

# 2. 매장정보 지도에 표시

## 2.1. 데이터 수집

### 2.1.1 데이터 파일 읽어오기

In [98]:
import pandas as pd

CB = pd.read_csv('김지훈-201835641-커피빈.csv', encoding='CP949', index_col=0, header=0, engine='python')

CB.head()  #작업 내용 확인용 출력

,store,address,phone
0,차병원점,서울시 강남구 논현로 566 강남차병원1층,02-538-7615
1,강남대로점,서울시 서초구 강남대로 369 1층,02-588-5778
2,청담에스점,"서울시 강남구 압구정로 461 네이처포엠빌딩B108,109호",02-548-6052
3,신사점,서울시 강남구 도산대로 126,02-548-2741
4,역삼점,"서울시 강남구 논현로 512 지상1,2층",02-569-8051


# 2.2 데이터 준비 및 탐색

### 2.2.1 시/도 행정구역 이름 정규화

In [99]:
addr = []

for address in CB.address:
    addr.append(str(address).split())

#작업 내용 확인용 출력
print('데이터 개수 : %d' % len(addr)) 
addr  

데이터 개수 : 49


[['서울시', '강남구', '논현로', '566', '강남차병원1층'],
 ['서울시', '서초구', '강남대로', '369', '1층'],
 ['서울시', '강남구', '압구정로', '461', '네이처포엠빌딩B108,109호'],
 ['서울시', '강남구', '도산대로', '126'],
 ['서울시', '강남구', '논현로', '512', '지상1,2층'],
 ['서울시', '서초구', '강남대로', '213', '24호', '지하1층'],
 ['서울시', '강남구', '삼성로', '716', 'LEE76빌딩2층'],
 ['서울', '서초구', '반포동', '736-17', 'P빌딩', '2층'],
 ['서울시', '강남구', '언주로', '30길', '10,112', '현대비젼21', '112호'],
 ['서울시', '강남구', '선릉로', '749', '1,2층'],
 ['서울시', '강남구', '도산대로49길', '13', '1층', '17,18호'],
 ['서울시', '서초구', '강남대로', '51길', '1', '511', 'TOWER', '1층'],
 ['서울시', '강남구', '논현', '231-13호', '팍스타워지하1층'],
 ['서울', '강남구', '테헤란로87길', '46', '지하', '2층'],
 ['서울시', '강남구', '영동대로', '511', '트레이드타워', '지하1층'],
 ['서울시', '강남구', '영동대로', '607', '1,2층'],
 ['서울시', '송파구', '석촌호수로', '118', '1층'],
 ['서울시', '서초구', '서초동', '1685-8호', '101~2호,113~4호,121호'],
 ['서울시', '강남구', '논현로', '717', '1층'],
 ['서울시', '서초구', '서초대로74길', '11', '지하2층'],
 ['서울시', '서초구', '서초대로74길', '4', '삼성생명보험서초타워내지하1층'],
 ['서울특별시', '서초구', '방배중앙로', '187'],
 ['서울시',

### - addr에서 행정구역 표준 이름이 아닌것 수정하기

In [100]:
addr2 = []

# addr에서 행정구역 표준 이름이 아닌것 수정하기
for i in range(len(addr)):
    if addr[i][0] == "서울": addr[i][0]="서울특별시"
    elif addr[i][0] == "서울시": addr[i][0]="서울특별시"
    elif addr[i][0] == "부산시": addr[i][0]="부산광역시"
    elif addr[i][0] == "인천": addr[i][0]="인천광역시"
    elif addr[i][0] == "광주": addr[i][0]="광주광역시"
    elif addr[i][0] == "대전시": addr[i][0]="대전광역시"
    elif addr[i][0] == "울산시": addr[i][0]="울산광역시"    
    elif addr[i][0] == "세종시": addr[i][0]="세종특별자치시"
    elif addr[i][0] == "경기": addr[i][0]="경기도"
    elif addr[i][0] == "충북": addr[i][0]="충청북도"
    elif addr[i][0] == "충남": addr[i][0]="충청남도"
    elif addr[i][0] == "전북": addr[i][0]="전라북도"
    elif addr[i][0] == "전남": addr[i][0]="전라남도"
    elif addr[i][0] == "경북": addr[i][0]="경상북도"
    elif addr[i][0] == "경남": addr[i][0]="경상남도"
    elif addr[i][0] == "제주": addr[i][0]="제주특별자치도"
    elif addr[i][0] == "제주도": addr[i][0]="제주특별자치도"
    elif addr[i][0] == "제주시": addr[i][0]="제주특별자치도"                                
       
    addr2.append(' '.join(addr[i]))  

addr2 #작업 내용 확인용 출력

['서울특별시 강남구 논현로 566 강남차병원1층',
 '서울특별시 서초구 강남대로 369 1층',
 '서울특별시 강남구 압구정로 461 네이처포엠빌딩B108,109호',
 '서울특별시 강남구 도산대로 126',
 '서울특별시 강남구 논현로 512 지상1,2층',
 '서울특별시 서초구 강남대로 213 24호 지하1층',
 '서울특별시 강남구 삼성로 716 LEE76빌딩2층',
 '서울특별시 서초구 반포동 736-17 P빌딩 2층',
 '서울특별시 강남구 언주로 30길 10,112 현대비젼21 112호',
 '서울특별시 강남구 선릉로 749 1,2층',
 '서울특별시 강남구 도산대로49길 13 1층 17,18호',
 '서울특별시 서초구 강남대로 51길 1 511 TOWER 1층',
 '서울특별시 강남구 논현 231-13호 팍스타워지하1층',
 '서울특별시 강남구 테헤란로87길 46 지하 2층',
 '서울특별시 강남구 영동대로 511 트레이드타워 지하1층',
 '서울특별시 강남구 영동대로 607 1,2층',
 '서울특별시 송파구 석촌호수로 118 1층',
 '서울특별시 서초구 서초동 1685-8호 101~2호,113~4호,121호',
 '서울특별시 강남구 논현로 717 1층',
 '서울특별시 서초구 서초대로74길 11 지하2층',
 '서울특별시 서초구 서초대로74길 4 삼성생명보험서초타워내지하1층',
 '서울특별시 서초구 방배중앙로 187',
 '서울특별시 강남구 테헤란로 20길 10 쓰리엠타워1층',
 '서울특별시 강남구 테헤란로 70길 12 1층',
 '서울특별시 강남구 봉은사로 628 엘슨빌딩1층',
 '서울특별시 송파구 오금로 11길 7',
 '서울특별시 강남구 논현로 38길 42 418호',
 '서울특별시 서초구 서초중앙로 43 로얄타워1층',
 '서울특별시 강남구 테헤란로4길 28 826-28호 1층',
 '서울특별시 강남구 테헤란로87길 36 공항타워 1층',
 '서울특별시 강남구 도산대로 67길 7,파크빌딩 1층',
 '서울특별시 종로구 서린동 13

In [101]:
addr2 = pd.DataFrame(addr2, columns=['address2'])

addr2 #작업 내용 확인용 출력

,address2
0,서울특별시 강남구 논현로 566 강남차병원1층
1,서울특별시 서초구 강남대로 369 1층
2,"서울특별시 강남구 압구정로 461 네이처포엠빌딩B108,109호"
3,서울특별시 강남구 도산대로 126
4,"서울특별시 강남구 논현로 512 지상1,2층"
5,서울특별시 서초구 강남대로 213 24호 지하1층
6,서울특별시 강남구 삼성로 716 LEE76빌딩2층
7,서울특별시 서초구 반포동 736-17 P빌딩 2층
8,"서울특별시 강남구 언주로 30길 10,112 현대비젼21 112호"
9,"서울특별시 강남구 선릉로 749 1,2층"


In [102]:
CB2 = pd.concat([CB, addr2],  axis=1 )

CB2.head()  #작업 내용 확인용 출력

,store,address,phone,address2
0,차병원점,서울시 강남구 논현로 566 강남차병원1층,02-538-7615,서울특별시 강남구 논현로 566 강남차병원1층
1,강남대로점,서울시 서초구 강남대로 369 1층,02-588-5778,서울특별시 서초구 강남대로 369 1층
2,청담에스점,"서울시 강남구 압구정로 461 네이처포엠빌딩B108,109호",02-548-6052,"서울특별시 강남구 압구정로 461 네이처포엠빌딩B108,109호"
3,신사점,서울시 강남구 도산대로 126,02-548-2741,서울특별시 강남구 도산대로 126
4,역삼점,"서울시 강남구 논현로 512 지상1,2층",02-569-8051,"서울특별시 강남구 논현로 512 지상1,2층"


In [103]:
CB2.to_csv('김지훈-201835641-커피빈-2.csv',encoding='CP949', index = False)

# 3. 데이터 모델링

### - 지도 정보 시각화 라이브러리 설치 및 임포트하기

In [104]:
!pip install folium

In [105]:
import folium

### 1) 가천대 좌표를 사용하여 지도 객체 테스트하기

In [106]:
map_osm = folium.Map(location=[37.4507128, 127.1288495], zoom_start = 16)

In [107]:
map_osm.save('map_gachon.html')

### 2) 정리해둔 김지훈-201835641-커피빈-2.csv 파일 로드

In [108]:
CB_file = pd.read_csv('김지훈-201835641-커피빈-2.csv',encoding='cp949',  engine='python')

CB_file.head() #작업 내용 확인용 출력

,store,address,phone,address2
0,차병원점,서울시 강남구 논현로 566 강남차병원1층,02-538-7615,서울특별시 강남구 논현로 566 강남차병원1층
1,강남대로점,서울시 서초구 강남대로 369 1층,02-588-5778,서울특별시 서초구 강남대로 369 1층
2,청담에스점,"서울시 강남구 압구정로 461 네이처포엠빌딩B108,109호",02-548-6052,"서울특별시 강남구 압구정로 461 네이처포엠빌딩B108,109호"
3,신사점,서울시 강남구 도산대로 126,02-548-2741,서울특별시 강남구 도산대로 126
4,역삼점,"서울시 강남구 논현로 512 지상1,2층",02-569-8051,"서울특별시 강남구 논현로 512 지상1,2층"


# 4. 지오코더를 사용해 위도 경도 정보 추가

In [109]:
!pip install requests

In [110]:
import requests
from urllib import parse
import pandas as pd

In [111]:
# API 요청.(본인의 API 키 사용)
def geocode(address, type='ROAD'):
    result = requests.get("http://api.vworld.kr/req/address?service=address"
        + "&request=getcoord" +"&version2.0" + "&crs=epsg:4326"
        + "&address=" + parse.quote(address) + "&type=" + type
        + "&key=EEF0CBC5-92EC-36C9-9333-84F617E823C8")
    return result

In [112]:
CB_file = pd.read_csv('김지훈-201835641-커피빈-2.csv',encoding='cp949',  engine='python')

CB_file.head() #작업 내용 확인용 출력

,store,address,phone,address2
0,차병원점,서울시 강남구 논현로 566 강남차병원1층,02-538-7615,서울특별시 강남구 논현로 566 강남차병원1층
1,강남대로점,서울시 서초구 강남대로 369 1층,02-588-5778,서울특별시 서초구 강남대로 369 1층
2,청담에스점,"서울시 강남구 압구정로 461 네이처포엠빌딩B108,109호",02-548-6052,"서울특별시 강남구 압구정로 461 네이처포엠빌딩B108,109호"
3,신사점,서울시 강남구 도산대로 126,02-548-2741,서울특별시 강남구 도산대로 126
4,역삼점,"서울시 강남구 논현로 512 지상1,2층",02-569-8051,"서울특별시 강남구 논현로 512 지상1,2층"


In [113]:
# 경도, 위도 필드 만들기
CB_file['경도'] = 0.0
CB_file['위도'] = 0.0
CB_file

,store,address,phone,address2,경도,위도
0,차병원점,서울시 강남구 논현로 566 강남차병원1층,02-538-7615,서울특별시 강남구 논현로 566 강남차병원1층,0.0,0.0
1,강남대로점,서울시 서초구 강남대로 369 1층,02-588-5778,서울특별시 서초구 강남대로 369 1층,0.0,0.0
2,청담에스점,"서울시 강남구 압구정로 461 네이처포엠빌딩B108,109호",02-548-6052,"서울특별시 강남구 압구정로 461 네이처포엠빌딩B108,109호",0.0,0.0
3,신사점,서울시 강남구 도산대로 126,02-548-2741,서울특별시 강남구 도산대로 126,0.0,0.0
4,역삼점,"서울시 강남구 논현로 512 지상1,2층",02-569-8051,"서울특별시 강남구 논현로 512 지상1,2층",0.0,0.0
5,양재스포타임점,서울시 서초구 강남대로 213 24호 지하1층,02-578-6833,서울특별시 서초구 강남대로 213 24호 지하1층,0.0,0.0
6,청담성당점,서울시 강남구 삼성로 716 LEE76빌딩2층,02-542-2053,서울특별시 강남구 삼성로 716 LEE76빌딩2층,0.0,0.0
7,영동점,서울 서초구 반포동 736-17 P빌딩 2층,02-3443-2096,서울특별시 서초구 반포동 736-17 P빌딩 2층,0.0,0.0
8,도곡점,"서울시 강남구 언주로 30길 10,112 현대비젼21 112호",02-572-2781,"서울특별시 강남구 언주로 30길 10,112 현대비젼21 112호",0.0,0.0
9,영동고앞점,"서울시 강남구 선릉로 749 1,2층",02-544-3794,"서울특별시 강남구 선릉로 749 1,2층",0.0,0.0


In [114]:
# 각 주소마다 API 요청
for i in range(len(CB_file)):
    result = None
    coord_x = coord_y = 0.0
    result = geocode(CB_file.at[i, 'address2']).json()
    if result['response']['status'] != 'NOT_FOUND':
        coord_x = result['response']['result']['point']['x']
        coord_y = result['response']['result']['point']['y']
    else:    # 도로명 주소로 못 찾을 시 지번 주소로 찾기
        result = geocode(CB_file.at[i, 'address2'], 'PARCEL').json()
        if result['response']['status'] != 'NOT_FOUND':
            coord_x = result['response']['result']['point']['x']
            coord_y = result['response']['result']['point']['y']
        else: 
            continue
    
    CB_file.at[i, '경도'] = round(float(coord_x), 4)
    CB_file.at[i, '위도'] = round(float(coord_y), 4)

In [115]:
CB_file

,store,address,phone,address2,경도,위도
0,차병원점,서울시 강남구 논현로 566 강남차병원1층,02-538-7615,서울특별시 강남구 논현로 566 강남차병원1층,127.0345,37.5068
1,강남대로점,서울시 서초구 강남대로 369 1층,02-588-5778,서울특별시 서초구 강남대로 369 1층,127.0282,37.4957
2,청담에스점,"서울시 강남구 압구정로 461 네이처포엠빌딩B108,109호",02-548-6052,"서울특별시 강남구 압구정로 461 네이처포엠빌딩B108,109호",127.0472,37.5252
3,신사점,서울시 강남구 도산대로 126,02-548-2741,서울특별시 강남구 도산대로 126,127.0224,37.5170
4,역삼점,"서울시 강남구 논현로 512 지상1,2층",02-569-8051,"서울특별시 강남구 논현로 512 지상1,2층",127.0365,37.5023
5,양재스포타임점,서울시 서초구 강남대로 213 24호 지하1층,02-578-6833,서울특별시 서초구 강남대로 213 24호 지하1층,127.0356,37.4824
6,청담성당점,서울시 강남구 삼성로 716 LEE76빌딩2층,02-542-2053,서울특별시 강남구 삼성로 716 LEE76빌딩2층,127.0500,37.5199
7,영동점,서울 서초구 반포동 736-17 P빌딩 2층,02-3443-2096,서울특별시 서초구 반포동 736-17 P빌딩 2층,127.0228,37.5068
8,도곡점,"서울시 강남구 언주로 30길 10,112 현대비젼21 112호",02-572-2781,"서울특별시 강남구 언주로 30길 10,112 현대비젼21 112호",127.0516,37.4873
9,영동고앞점,"서울시 강남구 선릉로 749 1,2층",02-544-3794,"서울특별시 강남구 선릉로 749 1,2층",127.0395,37.5215


In [116]:
# CB_file.to_csv('CB_geoAPI.shp_2024_2.csv',encoding='CP949', index = False)
CB_file.to_csv('CB_geoAPI.shp_김지훈-201835641-커피빈.csv',encoding='CP949', index = False)

### 지오코드 API를 사용하여 구한 GPS 좌표 파일 로드  ch9_geocodeAPI2024.ipynb 수행하여 좌표파일 작성

In [117]:
CB_geoData = pd.read_csv('CB_geoAPI.shp_김지훈-201835641-커피빈.csv',encoding='cp949',  engine='python')

len(CB_geoData) #확인용 출력

49

### map_CB 생성 (가천대 좌표)

In [118]:
map_CB = folium.Map(location=[37.4507128, 127.1288495], zoom_start = 15)

### for 반복문을 사용하여 CB_geoData 객체에 있는 매장 정보를 하나씩 읽음
### 매장에 대한 마커의 팝업 글자는 매장 이름으로 설정하고popup = store['store'] , 마커 모양은 파란색 깃발 모양으로 설정하여icon = 'flag' 마커를 만든 뒤folium.Marker( ) 지도 객체map_CB에 추가add_to(map_CB).

In [119]:
for i, store in CB_geoData.iterrows():
     folium.Marker(location = [store['위도'], store['경도']], popup =
                        store['store'], icon = folium.Icon(color = 'blue',
                        icon = 'flag')).add_to(map_CB)


In [120]:
map_CB.save('map_CBgeo_gachon.html')

In [121]:
import webbrowser

webbrowser.open('map_CBgeo_gachon.html')

True

In [122]:
map_CB